### Importing libraries

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer, DataCollatorForLanguageModeling, Trainer, TrainingArguments
from datasets import load_dataset
import torch

🚀 Using device: cuda
📥 Loading WikiText-2...
🔄 Tokenizing dataset...
🚀 Training GPT-2 (1 epoch, 50% WikiText-2, GPU)...


Step,Training Loss
100,4.003200
200,3.773100
300,3.712200
400,3.614900
500,3.582200
600,3.601500
700,3.559800
800,3.556700
900,3.480400
1000,3.534300


💾 Saving model...


('./gpt2-finetuned-wikitext2\\tokenizer_config.json',
 './gpt2-finetuned-wikitext2\\special_tokens_map.json',
 './gpt2-finetuned-wikitext2\\vocab.json',
 './gpt2-finetuned-wikitext2\\merges.txt',
 './gpt2-finetuned-wikitext2\\added_tokens.json')

### Check GPU

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


### Load 50% of WikiText-2

In [ ]:
print("Loading WikiText-2...")
dataset = load_dataset("wikitext", "wikitext-2-raw-v1", split="train[:50%]")

### Load GPT-2 tokenizer and model

In [ ]:

model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token  # Fix pad token error

model = GPT2LMHeadModel.from_pretrained(model_name).to(device)

### Tokenize dataset

In [ ]:

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=64)

print("🔄 Tokenizing dataset...")
tokenized_dataset = dataset.map(tokenize_function, batched=True, remove_columns=["text"])

### Data collator

In [ ]:

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

### Training arguments for GPU

In [ ]:

training_args = TrainingArguments(
    output_dir="./gpt2-finetuned-wikitext2",
    per_device_train_batch_size=8,
    num_train_epochs=1,
    save_steps=500,
    logging_steps=100,
    fp16=True, 
    overwrite_output_dir=True,
    report_to="none"
)


### Training model

In [ ]:

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=data_collator
)

print("Training GPT-2 (1 epoch, 50% WikiText-2, GPU)...")
trainer.train()

print("Saving model...")
model.save_pretrained("./gpt2-finetuned-wikitext2")
tokenizer.save_pretrained("./gpt2-finetuned-wikitext2")




### Taking prompt from the user

In [ ]:
from transformers import pipeline
import torch

device = 0 if torch.cuda.is_available() else -1

generator = pipeline("text-generation", model="./gpt2-finetuned-wikitext2", tokenizer="./gpt2-finetuned-wikitext2", device=device)

while True:
    prompt = input("Enter your prompt (or type 'exit'): ")
    if prompt.strip().lower() == "exit":
        break
    output = generator(prompt, max_length=100, num_return_sequences=1)
    print("\n GPT-2 Output:\n")
    print(output[0]['generated_text'])
    print("=" * 80)


Device set to use cuda:0
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.



🧠 GPT-2 Output:

The theory of relativity states that as light travels through space, it passes back and forth between two objects, each of which has several phases. The motion of matter occurs in the two phases of its motion, in the first phase and the second phase. After crossing both waves, the wave travels through its own phase of the velocity's motion and then back and forth between its two counterparts, before returning to the same phase once more. This symmetry in the motion of matter also makes it possible to predict
